In [1]:
import numpy as np
from qiskit import *
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import GroverOperator
import matplotlib
import random

In [34]:
def find_min(nums, iters):
    N = len(nums)
    t = nums[random.randint(0, N-1)]
    targets = [i for i in range(N) if nums[i]<=t ]
    for i in range(iters):
        new_t = nums[grover_search(targets, N)]
        if new_t<t:
            t=new_t
        targets = [target for target in targets if nums[target]<=t]
    return t
def grover_search(targets, N):
    d = int(np.ceil(np.log2(N)))
    oracle = np.identity(2**d)
    for target in targets:
        oracle[target, target] = -1
    oc = QuantumCircuit(d)
    oc.unitary(oracle, range(d))
    grover_op = GroverOperator(oc, insert_barriers=True)
    qc = QuantumCircuit(d)
    qc.h(range(d))
    r = int(np.sqrt(N/len(targets)))
    for i in range(1):
        qc = qc.compose(grover_op)
    qc.measure_all()
    backend = Aer.get_backend("qasm_simulator")
    qc.decompose().draw()
    job = execute(qc, backend)
    result = int(list(job.result().get_counts().keys())[0][::-1], 2)
    return result